# Aditep - Capstone Project

## 1. Introduction/Business Problem

A wine importer would like to open a distribution center in New York to distribute their import goods to local wine bars but they need to know which area the most of wine bars in New York are located so that they can minimise their delivery cost. 

## 2. Data

From Foursqure location data, I am to select venue category just only "Wine Bar" and then do clustering to select the most frequency cluster as a suitable area to open the distribution center.